# Gold Top Item Analytics Notebook

This notebook builds a gold-level analytics mart for top item views by platform and year. The workflow includes:
* Importing required libraries
* Loading and cleaning the silver event data from [workspace.silver_layer.silver_event](#table)
* Filtering for item view events and aggregating views per item/year
* Identifying the most used platform for each item/year
* Ranking items by popularity within each year
* Saving the final results to [workspace.gold_layer.gold_top_item](#table)

Use this notebook to analyze item popularity trends and platform usage over time.

### Import Libraries

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

### Define Function

### Load Data

In [0]:
# Load silver_event table from workspace.silver_layer without technical columns
technical_cols = ["tc_ingestion_timestamp", "tc_source_file", "tc_bronze_id"]
silver_event_df = spark.read.table("workspace.silver_layer.silver_event").drop(*technical_cols)

### EDA

In [0]:
# Basic analysis of "event_parameter_value" column
silver_event_df.groupBy("event_parameter_value").agg(
    F.count("*").alias("count"),
    F.countDistinct("event_name").alias("distinct_event_names"),
    F.min("event_year").alias("min_year"),
    F.max("event_year").alias("max_year")
).orderBy(F.desc("count")).display()

In [0]:
# Analysis of "event_parameter_name" and "event_parameter_value"
# 1️⃣ What are the values in "event_parameter_name"?
event_param_name_values_df = silver_event_df.groupBy("event_parameter_name").agg(
    F.count("*").alias("count"),
    F.countDistinct("event_parameter_value").alias("distinct_values")
).orderBy(F.desc("count"))
display(event_param_name_values_df)

# 2️⃣ For "event_parameter_name" == "item_id", are "event_parameter_value" values integer?
item_id_param_df = silver_event_df.filter(F.col("event_parameter_name") == "item_id")
item_id_type_df = item_id_param_df.withColumn(
    "is_integer", F.col("event_parameter_value").cast("int").isNotNull()
)
integer_check_df = item_id_type_df.groupBy("is_integer").agg(F.count("*").alias("count"))
display(integer_check_df)

In [0]:
# Top Item Analytics from silver_event_df
# 1️⃣ Filter for item views only (event_name == 'view_item')
silver_event_df = silver_event_df.filter(
    (F.col("event_name") == "view_item") & (F.col("event_parameter_name") == "item_id")
)

silver_event_df = silver_event_df.withColumn("event_parameter_value",
    F.expr("try_cast(event_parameter_value as bigint)")
)

# 2️⃣ Total views per item per year
views_df = (
    silver_event_df.groupBy("event_parameter_value", "event_year")
      .agg(F.count("*").alias("total_views"))
      .withColumnRenamed("event_parameter_value", "item_id")
)

# 3️⃣ Most used platform per item/year
platform_counts = (
    silver_event_df.groupBy("event_parameter_value", "event_year", "event_platform")
      .agg(F.count("*").alias("platform_views"))
)

platform_window = (
    Window.partitionBy("event_parameter_value", "event_year")
          .orderBy(F.desc("platform_views"))
)

top_platform_df = (
    platform_counts
        .withColumn("rn", F.row_number().over(platform_window))
        .filter("rn = 1")
        .select(
            F.col("event_parameter_value").alias("item_id"),
            "event_year",
            "event_platform"
        )
)

# 4️⃣ Rank items within each year based on total views
rank_window = Window.partitionBy("event_year").orderBy(F.desc("total_views"))

ranked_df = views_df.withColumn("item_rank", F.rank().over(rank_window))

# 5️⃣ Join into final mart
top_item_df = (
    ranked_df.join(top_platform_df, ["item_id", "event_year"], "left")
)

### Save Table

In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS gold_layer")

# Save Gold table
top_item_df.write.mode("overwrite").saveAsTable("workspace.gold_layer.gold_top_item")